In [1]:
!pip install sumy
!pip install datasets
!pip install tqdm
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 2.7 MB/s 
     |████████████████████████████████| 10.1 MB 6.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21712 sha256=cf66071405dfcb5c4a57370683868117380b267e9a153e7791c530bf90760a72
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b62b30e82892b0c84b35606dc739ca36b366f72ae48100079b5d68d242286ce5
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=2136eeeba197964c5ac64cc29f7

In [2]:
import os
import json
import nltk
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = '/content/drive/MyDrive/nlp-project/data'
BILLSUM_DATA_PATH = os.path.join(DATA_PATH, 'billsum_v4_1')

In [5]:
from datasets import load_dataset

In [6]:
TRAIN_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_train_data_final_OFFICIAL.jsonl")
TEST_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_test_data_final_OFFICIAL.jsonl")

dataset = load_dataset("json", data_files={
    "train": TRAIN_DATA_FILE,
    "test": TEST_DATA_FILE,
})

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a257b7a118a3705/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from tqdm import tqdm
import evaluate

In [8]:
def preprocess_text(row):
    text = row['text']
    text = text.split('\n')
    text = list(map(lambda x: x.strip(), text))
    text = list(filter(lambda x: True if x else False, text))
    # text = list(filter(lambda x: False if 'TITLE' in x else True, text))
    # text = list(filter(lambda x: False if 'SECTION' in x else True, text))
    # text = list(filter(lambda x: False if 'SEC.' in x else True, text))
    text = " ".join(text)
    text = sent_tokenize(text)
    text = " ".join(text[:3])
    row['predicted'] = text
    return row

In [9]:
dataset['test'] = dataset['test'].map(preprocess_text)

  0%|          | 0/3269 [00:00<?, ?ex/s]

In [10]:
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evaluate_summaries(dataset, metric, batch_size=16, column_text="text", column_summary="summary"):
    metric.add_batch(predictions=dataset['predicted'], references=dataset[column_summary])
    score = metric.compute()
    return score

In [11]:
rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

score = evaluate_summaries(dataset['test'], rouge_metric)

In [12]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"LEAD1"])

,rouge1,rouge2,rougeL,rougeLsum
LEAD1,0.160273,0.09542,0.113271,0.113278
